## Probar selenium para navegar con incognito y con la libreria time espaciarle los requests

In [12]:
#Librerias
import re
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import time

In [2]:
# Funcion get
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    head = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/59.0.3071.115 Safari/537.36'}
    try:
        with closing(get(url, stream=True, headers=head)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

# Codigo final todo junto

In [18]:
ids = list(range(10000))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [13]:
ids = [27100,10270,57503,10469,38019,55358,1066,11111]

In [14]:
##Loopeo sobre los id de la URL
games = []

try:
    for id in ids:
        url = 'https://howlongtobeat.com/game.php?id='+str(id)
        raw_html = simple_get(url)
        html = BeautifulSoup(raw_html, 'html.parser')

        ## Titulo del juego
        titulo = html.find('div', {'class':"profile_header shadow_text"}).text.split('\n')
        titulo = titulo[1].rstrip()
            #Pasamos a dict
        titulo_dict = {'Titulo' : titulo}


        ## Rating
        det=html.find('div', {'class':'profile_details'})
        details = []
        for i, li in enumerate(det.select('li')):
            dicc = {i: li.text}
            details.append(li.text)
        rating = details[4][0:2]
            #Pasamos a dict
        rating_dict = {'Rating': rating}
        rating_dict


        ## Completion Times
        tiempos = html.find_all('li',class_=re.compile("short time_"))
            #Descompongo cada uno de los tiempos
        times = []
        for n in range(4):
            x = tiempos[n].select('div')
            x = str(x[0])
            x = x.replace('<div>', '')
            x = x.replace('</div>','')
            x = x.rstrip()
            times.append(x)
            #Creo un diccionario con la clave de cada tiempo
        times_dict = {'Main Story'    : times[0],
                      'Main + Extras' : times[1],
                      'Completionist' : times[2],
                      'All Styles'    : times[3]}


        ## Info del juego
        prof = html.find_all('div', {'class':'profile_info'})
            #Creo un diccionario con la info
        info_dict = {'Developer'   : prof[0].contents[2].replace('\n','').rstrip(),
                     'Publishers'  : prof[1].contents[2].replace('\n','').rstrip(),
                     'Playable On' : prof[2].contents[2].replace('\n','').rstrip(),
                     'Genres'      : prof[3].contents[2].replace('\n','').rstrip()}

        ## Junto Todos los diccionarios en uno solo

        games.append({**titulo_dict, **rating_dict, **times_dict, **info_dict})
        time.sleep(2) # Esperar dos segundos entre cada loop
except:
    pass

In [15]:
import pandas as pd
games_df = pd.DataFrame(games)

In [16]:
games_df

,All Styles,Completionist,Developer,Genres,Main + Extras,Main Story,Playable On,Publishers,Rating,Titulo
0,54½ Hours,175 Hours,Rockstar Studios,"Third-Person, Action, Adventure, Open World, S...",63 Hours,40 Hours,"PlayStation 4, Xbox One",Rockstar Games,94,Red Dead Redemption 2
1,102 Hours,171 Hours,CD Projekt RED,"Third-Person, Action, Adventure, Open World, R...",103 Hours,49½ Hours,"Linux, PC, PlayStation 4, Xbox One","CD Projekt, Warner Bros. Interactive Entertain...",96,The Witcher 3: Wild Hunt
2,71 Hours,102 Hours,Ubisoft Quebec,"Third-Person, Action, Open World, Role-Playing",66 Hours,33 Hours,"PC, PlayStation 4, Xbox One",Ubisoft,86,Assassin's Creed Odyssey
3,14½ Hours,20½ Hours,"Crystal Dynamics, Eidos Montreal, Nixxes Softw...","Third-Person, Action, Adventure, Platform, Sho...",15½ Hours,11½ Hours,"Linux, Mac, PC, PlayStation 3, PlayStation 4, ...",Square Enix,84,Tomb Raider (2013)
4,86 Hours,177 Hours,Nintendo EPD,"Third-Person, Action, Adventure, Open World",91½ Hours,46½ Hours,"Nintendo Switch, Wii U",Nintendo,94,The Legend of Zelda: Breath of the Wild
5,26 Hours,26 Hours,Toys for Bob,"Third-Person, Platform",--,--,PlayStation 4,Activision,92,Spyro Reignited Trilogy
6,13 Hours,22½ Hours,"2K Marin, Digital Extremes, 2K China, Arkane S...","First-Person, Shooter",14½ Hours,11 Hours,"Mac, PC, PlayStation 3, PlayStation 4, Xbox 36...","2K Games, D3 Publisher",79,BioShock 2


In [9]:
games_df = games_df[['Titulo','Main Story','Main + Extras','Completionist','All Styles','Rating','Developer', 'Publishers', 'Genres']]

In [10]:
games_df

,Titulo,Main Story,Main + Extras,Completionist,All Styles,Rating,Developer,Publishers,Genres
0,Red Dead Redemption 2,40 Hours,63 Hours,175 Hours,54½ Hours,94,Rockstar Studios,Rockstar Games,"Third-Person, Action, Adventure, Open World, S..."
1,The Witcher 3: Wild Hunt,49½ Hours,103 Hours,171 Hours,102 Hours,96,CD Projekt RED,"CD Projekt, Warner Bros. Interactive Entertain...","Third-Person, Action, Adventure, Open World, R..."
2,Assassin's Creed Odyssey,33 Hours,66 Hours,102 Hours,71 Hours,86,Ubisoft Quebec,Ubisoft,"Third-Person, Action, Open World, Role-Playing"
3,Tomb Raider (2013),11½ Hours,15½ Hours,20½ Hours,14½ Hours,84,"Crystal Dynamics, Eidos Montreal, Nixxes Softw...",Square Enix,"Third-Person, Action, Adventure, Platform, Sho..."
4,The Legend of Zelda: Breath of the Wild,46½ Hours,91½ Hours,177 Hours,86 Hours,94,Nintendo EPD,Nintendo,"Third-Person, Action, Adventure, Open World"
5,Spyro Reignited Trilogy,--,--,26 Hours,26 Hours,92,Toys for Bob,Activision,"Third-Person, Platform"
6,BioShock 2,11 Hours,14½ Hours,22½ Hours,13 Hours,79,"2K Marin, Digital Extremes, 2K China, Arkane S...","2K Games, D3 Publisher","First-Person, Shooter"


In [35]:
print(type(games))


<class 'list'>


# -------CODIGO ANTERIOR-------------

## Titulo y rating

In [9]:
#Titulo del juego
titulo = html.find('div', {'class':"profile_header shadow_text"}).text.split('\n')
titulo = titulo[1].rstrip()

In [10]:
#Pasamos a dict
titulo_dict = {'Titulo' : titulo}
titulo_dict

{'Titulo': 'The Witcher 3: Wild Hunt'}

In [11]:
#Detalles de rating, backlog, etc
det=html.find('div', {'class':'profile_details'})

In [12]:
det

<div class="profile_details">
<li>2.1K Playing</li>
<li>7K Backlogs</li>
<li>914 Replays</li>
<li>1% Retired
</li>
<li>96% Rating</li>
<li>5.7K Beat</li>
<div class="clear"></div>
</div>

In [60]:
# Rating
details = []
for i, li in enumerate(det.select('li')):
        dicc = {i: li.text}
        details.append(li.text)
rating = details[4][0:2]

In [61]:
#Pasamos a dict
rating_dict = {'Rating': rating}
rating_dict

{'Rating': '94'}

## Completion Times

In [5]:
html

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="css/main_77.css" rel="stylesheet"/>
<link href="css/desktop_77.css" media="screen and (min-width: 769px)" rel="stylesheet"/>
<link href="css/mobile_77.css" media="screen and (max-width: 769px)" rel="stylesheet"/>
<title>How long is Spyro Reignited Trilogy? - HLTB</title> <meta content="How long is Spyro Reignited Trilogy? HowLongToBeat has the answer. Create a backlog, submit your game times and compete with your friends!" name="description"/> <meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/>
<link href="img/icons/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="img/icons/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="img/icons/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="img/icons/apple-touch-icon-76x76.png" re

In [12]:
import re

In [9]:
#Tiempos
tiempos = html.find_all('li', {'class':'short time_100','class':'short time_60'})

In [19]:
tiempos = html.find_all('li',class_=re.compile("short time_"))

In [23]:
#Vemos como quedó
tiempos[2]

<li class="short time_60">
<h5>Completionist</h5>
<div>175 Hours </div>
</li>

In [71]:
#Descompongo cada uno de los tiempos
times = []
for i in range(4):
    x = tiempos[i].select('div')
    x = str(x[0])
    x = x.replace('<div>', '')
    x = x.replace('</div>','')
    x = x.rstrip()
    times.append(x)

IndexError: list index out of range

In [25]:
times

['175 Hours']

In [19]:
# Creo un diccionario con la clave de cada tiempo
times_dict = {'Main Story': times[0],
              'Main + Extras' : times[1],
              'Completionist' : times[2],
              'All Styles' : times[3]}
times_dict

{'All Styles': '102 Hours',
 'Completionist': '171 Hours',
 'Main + Extras': '103 Hours',
 'Main Story': '49½ Hours'}

## Informacion adicional del juego

In [20]:
# Info del juego
prof = html.find_all('div', {'class':'profile_info'})

Podemos acceder a cada uno de los elementos loopeando sobre el objeto prof. Por otro lado, mediante el atributo contents y el indice 2, podemos obtener el texto que estamos buscando para cada uno de los atributos del juego

In [44]:
#Creo un diccionario con la info
info_dict = {'Developer'   : prof[0].contents[2].replace('\n','').rstrip(),
             'Publishers'  : prof[1].contents[2].replace('\n','').rstrip(),
             'Playable On' : prof[2].contents[2].replace('\n','').rstrip(),
             'Genres'      : prof[3].contents[2].replace('\n','').rstrip()}

In [46]:
info_dict

{'Developer': 'CD Projekt RED',
 'Genres': 'Third-Person, Action, Adventure, Open World, Role-Playing',
 'Playable On': 'Linux, PC, PlayStation 4, Xbox One',
 'Publishers': 'CD Projekt, Warner Bros. Interactive Entertainment'}

In [47]:
#Para juntar los diccionarios

{**titulo_dict, **rating_dict, **times_dict, **info_dict}

{'All Styles': '102 Hours',
 'Completionist': '171 Hours',
 'Developer': 'CD Projekt RED',
 'Genres': 'Third-Person, Action, Adventure, Open World, Role-Playing',
 'Main + Extras': '103 Hours',
 'Main Story': '49½ Hours',
 'Playable On': 'Linux, PC, PlayStation 4, Xbox One',
 'Publishers': 'CD Projekt, Warner Bros. Interactive Entertainment',
 'Rating': '96',
 'Titulo': 'The Witcher 3: Wild Hunt'}

In [207]:
# Creo un diccionario con la info
keys = ['Developer', 'Publishers', 'Playable On', 'Genres']
info_dict = {}

for i in range(4):
    prof[i].contents[2]
prof[0] ## Developer
prof[1] ## Publishers
prof[2] ## Playable On
prof[3] ##Genres

<div class="profile_info">
<strong>
Genres:
</strong>
Third-Person, Action, Adventure, Open World, Role-Playing </div>

In [3]:
#Raw HTML
raw_html = simple_get('https://howlongtobeat.com/game.php?id=55358')

In [4]:
#Paso el request a un objeto de BeatifulSoup
html = BeautifulSoup(raw_html, 'html.parser')